In [1]:
%%html
<style>
.cell-output-ipywidget-background {
   background-color: transparent !important;
}
.jp-OutputArea-output {
   background-color: transparent;
}  
</style>

In [2]:
SERVER = 'srvdb04.seac.local'

In [3]:
import pyodbc

connection_string = f"""
    driver={'{ODBC Driver 17 for SQL Server}'};
    server={SERVER};
    trusted_connection=yes;
"""

conn_server = pyodbc.connect(connection_string)

In [4]:
import sqlite3
import os


if os.path.exists(".sqlite3"):
    os.remove(".sqlite3")

conn_database = sqlite3.connect(".sqlite3")

curr = conn_database.cursor()

create_tables = open(".create", "r").read()
curr.executescript(create_tables)

In [5]:
import sqlparse
select = open(".select", "r").read()
select = [sqlparse.format(j).replace(';', '') for j in sqlparse.split(select)]
tables = ['registry', 'callcenter', 'wages']
select = dict(zip(tables, select))

In [6]:
import pandas as pd
import warnings
from tqdm import notebook

warnings.filterwarnings('ignore', category=UserWarning)

for table, query in notebook.tqdm(select.items()):
    df = pd.read_sql(query, conn_server)
    df.to_sql(table, conn_database, if_exists="append", index=False)

  0%|          | 0/3 [00:00<?, ?it/s]